In [9]:
import numpy as np


In [10]:
pred_data = np.load('results\stock_000001SZ_50_1_PatchTST_stock_custom_ftMS_sl50_ll48_pl1_dm128_nh16_el3_dl1_df256_fc1_ebtimeF_dtTrue_Exp_0\pred.npy')
print(pred_data.shape)
pred_data = pred_data.reshape((pred_data.shape[0]))
print(pred_data)

(800, 1, 1)
[-2.31017873e-01 -1.07697271e-01  1.54686525e-01  1.08340219e-01
  1.73660114e-01  1.16317414e-01  9.80943516e-02 -2.41092354e-01
 -1.76354319e-01  2.03133561e-02  6.94016889e-02  8.56201202e-02
  2.26016864e-01  4.88781810e-01  1.57480657e-01  1.64953843e-01
 -2.62441188e-02 -1.49851859e-01  8.03399906e-02  9.46095586e-03
  6.90073743e-02  4.16994542e-02  3.83093059e-01  2.91345060e-01
  1.67324603e-01  1.90871805e-01 -1.57918185e-01  2.48295814e-01
 -8.10822379e-03 -1.54851243e-01  2.21086089e-02 -2.08796263e-01
  7.36464746e-03  7.29896501e-02  1.68870330e-01 -1.94054857e-01
 -2.53065377e-01  2.71778535e-02  8.49928334e-02  1.42651752e-01
  1.10371813e-01  1.42122224e-01  8.18477720e-02  8.87190849e-02
 -6.13708533e-02 -1.19570993e-01 -5.55876046e-02  4.57581878e-02
 -2.38876417e-02 -4.01575454e-02 -8.06425288e-02  1.12165965e-01
  2.14784890e-02  2.71548331e-01  1.91846907e-01  1.33425519e-02
 -5.48530892e-02  1.57391787e-01  1.64139688e-01 -1.36107013e-01
  2.16617286e

In [11]:
input_x = np.load('results\stock_000001SZ_50_1_PatchTST_stock_custom_ftMS_sl50_ll48_pl1_dm128_nh16_el3_dl1_df256_fc1_ebtimeF_dtTrue_Exp_0\\x.npy')
input_x_0 = input_x[0].astype(np.float32)
print(input_x_0.shape)
print(input_x_0[:5])
print(input_x_0.dtype)

A_x = np.genfromtxt('F:\program repo\Stock_models_transformer\PatchTST_supervised\stock_data\stock_000001SZ.csv', delimiter=',', skip_header=1)
A_x = A_x.astype(np.float32)
A_x = A_x[: , 1:]
print(A_x[:5])
print(A_x.shape)
print(A_x.dtype)

(50, 10)
[[ 0.0711517  -0.10713951 -0.05039163 -0.69033104  0.4214851  -1.0235416
  -1.0235437   0.4783517  -0.6047336  -1.2321233 ]
 [ 0.7564112   0.47436583 -0.41046858  0.6616228  -0.25556126 -0.4132469
  -0.41322967 -0.7200914  -0.40878552  0.5842195 ]
 [ 0.6341729   0.41219327  1.1319045   1.129228    1.3660156   1.3299525
   1.3299261   0.6637056   1.0929415   1.3433521 ]
 [-0.0952591  -0.21028593  1.0088725  -0.3925133   0.66663474  0.24728248
   0.24726728  0.66286093  0.04906634 -0.2502623 ]
 [ 0.41804662  0.19601369 -0.04902364 -0.20796102 -1.3714114  -1.5591776
  -1.559161    0.34893957 -0.02523217 -1.3096722 ]]
float32
[[ 1.4102225  -0.91394866 -0.98737097 -0.8517691   0.5642528  -0.08899137
   0.23311062  0.23310582 -1.7782385  -1.8612044 ]
 [-0.7934654  -0.91635233 -0.9879087   1.1337966   0.2360118   1.0127833
  -0.7671213  -0.7670756   0.36442605  0.27104446]
 [-0.03357297 -0.9590712  -1.005979   -0.6095104  -0.4015359   0.11412193
  -1.0977176  -1.0976702  -0.7587891  

In [12]:
def find_subset_pos(A, B):
    for i in range(A.shape[0] - B.shape[0] + 1):
        if np.array_equal(A[i:i+B.shape[0]], B):
            return i
    return -1

In [13]:
find_subset_pos(A_x, input_x_0)

-1

In [14]:
print(A_x[-800:])

[[-4.2595741e-01 -1.3548252e-01 -1.6420829e-01 ... -2.2232516e-01
  -2.2275068e-01 -2.9122788e-01]
 [ 5.1335007e-02  1.5913811e-01  1.2088777e-01 ...  1.2384400e-01
   1.3872205e-01  3.5338530e-01]
 [ 7.8551102e-01  1.0325847e+00  9.7070819e-01 ...  7.4775928e-01
  -2.1800448e-01  4.7153950e-01]
 ...
 [ 3.6301085e-01  3.2052347e-01  1.8120158e-01 ... -1.4411075e-01
  -3.4662324e-01 -3.7125474e-01]
 [-6.3768692e-02  4.3213534e-01  2.7824026e-01 ...  1.1496682e-01
  -3.7391346e-02  1.2121058e-01]
 [-4.2655730e-01  2.1643795e-01  7.7662624e-02 ... -6.9888145e-02
   2.1860100e-04 -1.9569957e-01]]


In [15]:
x_arc = np.load('x_arc.npy')
print(x_arc[0][:5])
print(input_x_0[:5])

[[ 0.3745726   0.18165718 -0.04974989 -0.65400213  0.38936687 -0.9564399
  -0.9564345   0.43012086 -0.5715391  -1.1851393 ]
 [ 1.1163275   0.8215877  -0.38438216  0.62026215 -0.241079   -0.39133394
  -0.3913124  -0.66950536 -0.38975364  0.5507004 ]
 [ 0.98401135  0.7531685   1.0489998   1.0609965   1.2688862   1.2227918
   1.2227678   0.6001915   1.0034324   1.2761871 ]
 [ 0.19444227  0.06814741  0.93466175 -0.3732984   0.61764306  0.22028716
   0.22027665  0.59941643  0.03500583 -0.24679588]
 [ 0.7500668   0.51526886 -0.04847856 -0.19935136 -1.2801262  -1.4524153
  -1.4523908   0.3113793  -0.03392257 -1.2592511 ]]
[[ 0.0711517  -0.10713951 -0.05039163 -0.69033104  0.4214851  -1.0235416
  -1.0235437   0.4783517  -0.6047336  -1.2321233 ]
 [ 0.7564112   0.47436583 -0.41046858  0.6616228  -0.25556126 -0.4132469
  -0.41322967 -0.7200914  -0.40878552  0.5842195 ]
 [ 0.6341729   0.41219327  1.1319045   1.129228    1.3660156   1.3299525
   1.3299261   0.6637056   1.0929415   1.3433521 ]
 [-0.